In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
import pandas
import sqlite3

from sklearn import *

/home/riri/.virtualenvs/mxnet-3/lib/python3.4/site-packages/sklearn/lda.py:4: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWarning)
/home/riri/.virtualenvs/mxnet-3/lib/python3.4/site-packages/sklearn/qda.py:4: DeprecationWarning: qda.QDA has been moved to discriminant_analysis.QuadraticDiscriminantAnalysis in 0.17 and will be removed in 0.19.
  "in 0.17 and will be removed in 0.19.", DeprecationWarning)


In [4]:
train_data_sample = pandas.read_sql(
    'select * from data where adjusted_demand is not null order by random() limit 10000', 
    con=sqlite3.connect('./data.sqlite3')).drop(['id'], axis=1)
test_data_sample = pandas.read_sql(
    'select * from data where adjusted_demand is null order by random() limit 10000', 
    con=sqlite3.connect('./data.sqlite3')).drop(['id'], axis=1)

In [5]:
drop_cols = ['week_sales', 'week_returns', 'next_week_returns', 'next_week_returns_peso', 'adjusted_demand']
X_test = test_data_sample.drop(drop_cols, axis=1)
X_train = train_data_sample.drop(drop_cols, axis=1)
X = pandas.concat([X_test, X_train])
y = pandas.Series(np.concatenate([
        np.ones(X_test.shape[0]),
        np.zeros(X_train.shape[0])
    ]))

# Week number

The week number seems to be highly informative - this is because the dataset is a time series, and we're trying to predict several weeks of the dataset.

In [6]:
cls = ensemble.ExtraTreesClassifier(n_estimators=1000)
cls.fit(X, y)
cross_validation.cross_val_score(cls, X, y)

array([ 1.,  1.,  1.])

In [7]:
pandas.Series(cls.feature_importances_, index=X.columns)

week_num         0.980745
sales_depo       0.004345
sales_channel    0.000768
route_id         0.004226
client_id        0.004129
product_id       0.005786
dtype: float64

# Other parts

In [8]:
cls = ensemble.ExtraTreesClassifier(n_estimators=1000)
cls.fit(X.ix[:, 1:], y)
cross_validation.cross_val_score(cls, X.ix[:, 1:], y)

array([ 0.53059388,  0.52880288,  0.53630363])

In [9]:
cls.feature_importances_

array([ 0.21629634,  0.00716313,  0.26501528,  0.28106206,  0.23046319])

In [10]:
pandas.Series(cls.feature_importances_, index=X.ix[:, 1:].columns)

sales_depo       0.216296
sales_channel    0.007163
route_id         0.265015
client_id        0.281062
product_id       0.230463
dtype: float64

In [28]:
train_test_depos = pandas.read_sql(
    '''
    select distinct 'train' as grp, sales_depo from data where adjusted_demand is not null UNION
    select distinct 'test' as grp, sales_depo from data where adjusted_demand is null
    ''', 
    con=sqlite3.connect('./data.sqlite3'))

In [29]:
all(train_test_depos.groupby('grp').sales_depo.unique()['test'] == 
    train_test_depos.groupby('grp').sales_depo.unique()['train'])

True

In [30]:
train_test_weeks = pandas.read_sql(
    '''
    select distinct 'train' grp, week_num from data where adjusted_demand is not null UNION
    select distinct 'test' grp, week_num from data where adjusted_demand is null
    ''', 
    con=sqlite3.connect('./data.sqlite3'))

In [33]:
train_test_weeks.sort_values('week_num')

,grp,week_num
2,train,3
3,train,4
4,train,5
5,train,6
6,train,7
7,train,8
8,train,9
0,test,10
1,test,11
